# Расчет проводимости трубопроводов с помощью МУК

In [1]:
import os
import math
import numpy as np
from graph import Graph #собственный класс для графиков
from matplotlib import pyplot as plt

## Прямоугольное сечение

### Угловые коэффициенты

In [2]:
def scalar_prod(v_1, v_2):
    v_1 , v_2 = np.array(v_1), np.array(v_2)
    return (v_1*v_2).sum()

def module(v):
    return math.sqrt( scalar_prod(v, v) )

def elementary(center_i : list, center_j : list, normal_i : list, normal_j : list, F_j):
    #F_j - площадь одной ячейки коллектора.
    center_i, center_j = np.array(center_i), np.array(center_j)
    
    r        = module(center_i - center_j)
    n_i, n_j = module(normal_i), module(normal_j)
    
    cos_1 = abs( scalar_prod( normal_i, center_j - center_i ) ) / (r*n_i)
    cos_2 = abs( scalar_prod( normal_j, center_i - center_j ) ) / (r*n_j)
    return cos_1*cos_2/(math.pi*r**2)*F_j

In [3]:
def local(center_i, centers_2, normal_i, normal_j, F_j):
    return sum([ elementary(center_i, center, normal_i, normal_j, F_j) for center in centers_2])

def emitter_to_collector(centers_1, centers_2,normal_1, normal_2, F_i, F_j):
    #F_i - площадь одной ячейки эмиттера.
    return sum( [ local(center, centers_2, normal_1, normal_2, F_j) for center in centers_1 ] ) * F_i

### Разбиение

In [4]:
def euclid(num_1, num_2):
    while num_1 != num_2:
        if num_1 > num_2:
            num_1 -= num_2
        else:
            num_2 -= num_1
    return num_1

#ф-ция находит минимальное количество квадратов
#с одинаковой площадью, на которое можно разбить
#данный прямоугольник
def search_squares(a : int, b : int) -> int:
    gcd = euclid(a,b) #НОД
    return a*b/gcd

#ф-ция ищет оптимальное кол-во квадратов 
#в зависимости от введенного кол-ва
def optimal_squares(a, b, num):
    min_num = search_squares(a,b)
    n       = num//(4*min_num)
    return min_num if n == 0 else 4*min_num*n

In [5]:
#функции разбиения каждой отдельной поверхности на ячейки.
#1-входное, 2-выходное сечения.
#3-верхняя, 5-нижняя грани.
#если смотреть в направлении от 1 к 2, то
#3,4,5 и 6 грани пронумерованы по часовой стрелке.
def breaking_1(a, b, num, s = 0) -> list:
    #a - вдоль x, b - вдоль z, s - вдоль y
    res = []
    num = optimal_squares(a,b,num)
    
    square_side = math.sqrt(a*b/num)
    l_1 = int(a/square_side)
    l_2 = int(b/square_side)
    for i in range(l_1):
        x = a/(2*l_1) + a/l_1*i
        for j in range(l_2):
            z = b/(2*l_2) + b/l_2*j
            center = [x,0,z]
            res.append(center)
    return res, num

def breaking_2(a, b, num, s) -> list:
    res = []
    num = optimal_squares(a,b,num)
    
    square_side = math.sqrt(a*b/num)
    l_1 = int(a/square_side)
    l_2 = int(b/square_side)
    for i in range(l_1):
        x = a/(2*l_1) + a/l_1*i
        for j in range(l_2):
            z = b/(2*l_2) + b/l_2*j
            center = [x,s,z]
            res.append(center)
    return res, num

def breaking_3(a, b, num, s) -> list:
    res = []
    num = optimal_squares(a,s,num)
    
    square_side = math.sqrt(a*s/num)
    l_1 = int(a/square_side)
    l_2 = int(s/square_side)
    for i in range(l_1):
        x = a/(2*l_1) + a/l_1*i
        for j in range(l_2):
            y = s/(2*l_2) + s/l_2*j
            center = [x,y,b]
            res.append(center)
    return res, num

def breaking_4(a, b, num, s) -> list:
    res = []
    num = optimal_squares(s,b,num)
    
    square_side = math.sqrt(s*b/num)
    l_1 = int(s/square_side)
    l_2 = int(b/square_side)
    for i in range(l_1):
        y = s/(2*l_1) + s/l_1*i
        for j in range(l_2):
            z = b/(2*l_2) + b/l_2*j
            center = [0,y,z]
            res.append(center)
    return res, num

def breaking_5(a, b, num, s) -> list:
    res = []
    num = optimal_squares(a,s,num)
    
    square_side = math.sqrt(a*s/num)
    l_1 = int(a/square_side)
    l_2 = int(s/square_side)
    for i in range(l_1):
        x = a/(2*l_1) + a/l_1*i
        for j in range(l_2):
            y = s/(2*l_2) + s/l_2*j
            center = [x,y,0]
            res.append(center)
    return res, num

def breaking_6(a, b, num, s) -> list:
    res = []
    num = optimal_squares(s,b,num)
    
    square_side = math.sqrt(s*b/num)
    l_1 = int(s/square_side)
    l_2 = int(b/square_side)
    for i in range(l_1):
        y = s/(2*l_1) + s/l_1*i
        for j in range(l_2):
            z = b/(2*l_2) + b/l_2*j
            center = [a,y,z]
            res.append(center)
    return res, num

In [6]:
#площади "уникальных" поверхностей - area_1 == area_2 итд
def area_1(a, b, s):
    return a*b

def area_3(a, b, s):
    return a*s

def area_4(a, b, s):
    return s*b

### Проводимость

In [7]:
class Rectangular:
    def __init__(self, a, b, L):
        self.a = a
        self.b = b
        self.L = L
        self.normals  = [ [0,1,0], [0,1,0], [0,0,1], [1,0,0], [0,0,1], [1,0,0] ]
        self.breaking = [ breaking_1, breaking_2, breaking_3, breaking_4, breaking_5, breaking_6 ]
        self.areas    = [ area_1, area_1, area_3, area_4, area_3, area_4 ]
    
    def some_pair(self, i, j, num_1, num_2):
        #i - номер эмиттера   от 1 до 6.
        #j - номер коллектора от 1 до 6.
        #num_1, num_2 - на сколько частей разбить
        #эмиттер и коллектор соответсвенно.
        i-=1 #это - номер эмиттера   в массивах.
        j-=1 #это - номер коллектора в массивах.
        
        #параметры коллектора:
        #num_j - число квадратов на которое 
        #удалось разделить коллектор в итоге
        collector, num_j = self.breaking[j](self.a,self.b,num_2,self.L)
        normal_j= self.normals[j]
        cell_j  = self.areas[j](self.a,self.b,self.L)/num_j
        
        #параметры эмиттера:
        #num_i - число квадратов на которое 
        #удалось разделить эмиттер в итоге
        emitter, num_i = self.breaking[i](self.a,self.b,num_1,self.L)
        normal_i= self.normals[i]
        area_i  = self.areas[i](self.a,self.b,self.L)
        cell_i  = area_i/num_i
        return emitter_to_collector(emitter, normal_i, collector, normal_j, cell_i, cell_j)/area_i
    
    def matrix(self, num_1, num_2):
        res  = np.array([])    #будущая матрица УК
        line = np.array([0]*6) #текущий столбец матрицы
        #т.к. УК вида phi_21, phi_31, ..., phi_61
        #не используются в дальнейшем
        res  = np.append(res, line)
        line = np.array([0]*6)
        for j in range(2,5):
            #благодаря симметрии УК для граней 5 и 6
            #в качестве коллекторов можно не считать,
            #симметрия учтена при решении системы ур-ний 2.1
            res  = np.append(res, line)
            line = np.array([])
            for i in range(7):
                #чтобы phi_ij = matrix[j][i]
                if i:
                    line = np.append(line, self.some_pair(i,j,num_1,num_2))
                else:
                    line = np.array([0])
                    
        #транспонируем, чтобы можно было 
        #найти phi_ij = matrix[i][j],
        #а не  phi_ij = matrix[j][i]
        return np.transpose(res)
    
    def clausing(self, num_1 = 10, num_2 = 10):
        phi = self.matrix(num_1, num_2)
        
        q_3 = phi[1][3] + phi[1][4]/(1-phi[6][4])
        q_3/= 1 - phi[5][3] - 2*phi[4][3]*2*phi[3][4]/(1-phi[6][4])
        
        q_4 = phi[1][4]+2*phi[3][4]*q_3
        q_4/= 1 - phi[6][4]
        
        res = phi[1][2]
        res+= 2*phi[3][2]*q_3
        res+= 2*phi[4][2]*q_4
        return res

### Тестирование

In [8]:
#тесты на сами УК
def test_1(t, num_1 = 10, num_2 = 10):
    #вывод всех пар УК
    phi = t.matrix(num_1, num_2)
    for i in range(1,len(phi)):
        for j in range(1,len(phi[i])):
            print("phi_" + str(i)+str(j) + " =", phi[i][j])
    return

def test_2(t, num_1 = 10, num_2 = 10):
    #меньше ли 1
    return t.clausing(l_1,l_2) <= 1

def test_3(t, num_1 = 10, num_2 = 10):
    #больше ли 0
    return t.clausing(l_1,l_2) >= 0

In [9]:
tube1, tube_2 = Rectangular(1,1,1), Rectangular(1,5,1)

In [10]:
test_1(tube1)
test_1(tube_2)

NameError: name 'l_2' is not defined

In [ ]:
test_2(c_1)
test_2(c_2)

In [ ]:
test_3(c_1)
test_3(c_2)

In [ ]:
def angles(num = 5):
    #зависимость элементарного УК от угла
    c_i = [1/2, 0, 1/2]
    c_j = [1/2, 1, 1/2]
    n_i = [0,1,0]
    angle_ar = np.linspace(0,1,num)*math.pi
    
    x, y = np.sin(angle_ar), np.cos(angle_ar)
    Y    = []
    for i in range(num):
        n_j = [x[i], y[i], 0]
        X   = angle_ar/math.pi*180
        Y.append(elementary(c_i,c_j,n_i,n_j,0.05))
    
    graph = Graph()
    graph.customize_graph("Зависимость  элем-го УК от угла", "Угол в градусах")
    plt.plot(X,Y, marker="o")
    graph.save("angle.png", "tests")
    return

def distance(num = 5):
    #зависимость  элем-го УК от расстояния
    c_i  = [1/2, 0, 1/2]
    n_i  = [0,1,0]
    n_j  = [np.sin(math.pi/4), np.cos(math.pi/4),0]
    dist = np.linspace(0,10,num)
    Y    = []
    for d in dist:
        c_j = c_j = [1/2, d, 1/2]
        Y.append(elementary(c_i,c_j,n_i,n_j,0.05))
        
    graph = Graph()
    graph.customize_graph("Зависимость  элем-го УК от расстояния", "Расстояние")
    plt.plot(dist,Y, marker="o")
    graph.save("distance.png", "tests")
    return

In [ ]:
angles(50)

In [ ]:
distance(20)